In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle as pkl
import subprocess
import pandas as pd
from pathlib import Path

import napari
from magicgui import magicgui

from multiplex_pipeline.core_selection.viewer_utils import display_saved_rois, save_rois_from_viewer, redo_cores_layer, redo_bbox_layer
from multiplex_pipeline.utils.utils import change_to_wsl_path, load_analysis_settings, load_workstation_config
from multiplex_pipeline.im_utils import get_org_im_shape, prepare_rgb_image
from multiplex_pipeline.roi_utils import read_in_saved_rois, get_refined_rectangles, get_visual_rectangles, prepare_poly_df_for_saving, xywh_to_corners

## Read in config

In [ ]:
# load pipeline configuration
config = load_workstation_config(r'R:\Wayne\BLCA\pipeline_settings.yaml')
config

{'model_path': '/mnt/d/data_analysis/2024_bladder/sam2',
 'sam_env': '/home/kasia/miniforge3/envs/sam2-env/bin/python'}

In [ ]:
# load analysis configuration
settings_path = r'R:\Wayne\BLCA\BLCA-1B_Analysis\analysis_settings_BLCA1B.yaml'

settings = load_analysis_settings(settings_path)
settings

{'image_path': 'R:/CellDive/BLCA-1B/BLCA-1B_Final/BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_path': 'R:/Wayne/BLCA/BLCA-1B_Analysis/cores.csv',
 'temp_dir_cores': 'D:/multiplex_pipeline/tests',
 'output_dir_cores': 'R:/Wayne/BLCA/BLCA-1B_Analysis/temp',
 'include_channels': [],
 'exclude_channels': [],
 'use_channels': [],
 'core_detection': {'im_level': 6,
  'min_area': 2000,
  'max_area': 10000,
  'min_iou': 0.8,
  'min_st': 0.9,
  'min_int': 15,
  'frame': 4}}

In [4]:
# simplify variable names
im_level = settings['core_detection']['im_level']
save_path = settings['core_info_path']

In [5]:
# ensure that the directory to save core info exists
os.makedirs(os.path.dirname(settings['core_info_path']), exist_ok=True)

## Read in data

In [6]:
# read in image data

# get the original image shape
org_im_shape = get_org_im_shape(settings['image_path'])

# prepare the image for display
im_rgb = prepare_rgb_image(settings['image_path'], req_level = im_level)
im = im_rgb[:, :, 0]
im_rgb.shape

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


(1072, 558, 3)

## Create a viewer with options

In [7]:
viewer = napari.Viewer()
viewer.add_image(im)

# add a red rectangle to frame the image
frame_rect = xywh_to_corners([0,0,im.shape[1],im.shape[0]], frame = 0)
viewer.add_shapes(frame_rect, edge_color='white', face_color = 'transparent', shape_type='rectangle', edge_width=2, name = 'frame')

# add a layer for the saved rois
display_saved_rois(viewer, IM_LEVEL = im_level, save_path = save_path)


#########################################################################################
# add diplay saved button
@magicgui(auto_call=False, call_button="Display Saved Cores", result_widget=False)
def display_saved_rois_button(viewer: napari.Viewer):
    display_saved_rois(viewer, IM_LEVEL = im_level, save_path = save_path)  

# Add widget to viewer
viewer.window.add_dock_widget(display_saved_rois_button, area='left')

#########################################################################################
# add a save button
@magicgui(auto_call=False, call_button="Save Cores")  
def save_button(viewer: napari.Viewer):
    save_rois_from_viewer(viewer, org_im_shape = org_im_shape, req_level = im_level, save_path = save_path)

# Add widget to viewer
viewer.window.add_dock_widget(save_button, area='left')

## Get suggestions for core detection from the SAM2 model

Execute the cell below to get suggestions from SAM model about the cores position. 

In [8]:
# Linux path to the image
im_wsl = change_to_wsl_path(settings['image_path'])

# temporary output path for masks
out_win = os.path.join(os.getcwd(), 'masks.pkl')
out_wsl = change_to_wsl_path(out_win)

# Combine the command
command = [
    "wsl",
    config['sam_env'],  
    "-m", "multiplex_pipeline.core_selection.suggest_cores",
    im_wsl, 
    str(im_level),
    config['model_path'],
    out_wsl
]

# Execute the script
process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,  # Pipe the standard output
    stderr=subprocess.PIPE,  # Pipe the standard error
    text=True                # Capture output as text
)

# Display output in real time
while True:
    output = process.stdout.readline()
    if output == "" and process.poll() is not None:
        break
    if output:
        print(output, end="")  # Print each line as it becomes available

# Print any remaining errors
errors = process.stderr.read()
if errors:
    print(f"ERROR: {errors}")

# load the masks
masks = pkl.load(open(out_win, 'rb'))
print(f'Initial number of masks: {len(masks)}')

# remove the masks file
os.remove(out_win)

# refine masks
rect_list = get_refined_rectangles(masks, im = im_rgb[:,:,0], 
                                   frame = settings['core_detection']['frame'], 
                                   min_area = settings['core_detection']['min_area'],
                                   max_area = settings['core_detection']['max_area'],
                                   min_iou = settings['core_detection']['min_iou'],
                                   min_stability = settings['core_detection']['min_st'],
                                   min_int = settings['core_detection']['min_int']
                                   )
print('Number of refined masks:', len(rect_list))

# prepare df
df = prepare_poly_df_for_saving(rect_list, poly_types = ['rectangle']*len(rect_list), org_im_shape = org_im_shape, req_level = im_level)
rect_list = get_visual_rectangles(df, im_level)
poly_list = [(x/(2**im_level)).astype('int') for x in df.polygon_vertices.to_list()]

# add to napari
redo_cores_layer(viewer,rect_list,shape_type = df.poly_type.to_list())
redo_bbox_layer(viewer,rect_list,df['core_name'].tolist())

Input image: /mnt/r/CellDive/BLCA-1B/BLCA-1B_Final/BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif
Results will be saved to: /mnt/d/multiplex_pipeline/notebooks/masks.pkl

Preparing RGB image for segmentation...
Segmenting image. It should take around 1 min. Started at 2025-09-22 12:05:05...
Saving masks...
Initial number of masks: 101
Number of refined masks: 49
